In [1]:
import os
if "ntbk" in os.getcwd():
    os.chdir("..")
print(os.getcwd())

import sys
sys.path.append(os.path.join(os.getcwd(), "otgnn"))


%load_ext autoreload
%autoreload 2

/home/octav/gitrepos/tum-thesis


In [20]:
from otgnn.models import GCN
from otgnn.graph import MolGraph

from mol_opt.data_mol_opt import MolOptDataset
from mol_opt.data_mol_opt import get_loader
from mol_opt.arguments import get_args

from rdkit.Chem import MolFromSmiles

import torch

In [3]:
sys.path

['/home/octav/anaconda3/envs/mol_ot/lib/python36.zip',
 '/home/octav/anaconda3/envs/mol_ot/lib/python3.6',
 '/home/octav/anaconda3/envs/mol_ot/lib/python3.6/lib-dynload',
 '',
 '/home/octav/anaconda3/envs/mol_ot/lib/python3.6/site-packages',
 '/home/octav/anaconda3/envs/mol_ot/lib/python3.6/site-packages/IPython/extensions',
 '/home/octav/.ipython',
 '/home/octav/gitrepos/tum-thesis/otgnn']

In [4]:
data_loader = get_loader("iclr19-graph2graph/data/qed", 48, True)

In [5]:
for i in data_loader:
    X = (MolGraph(i[0]))
    Y = (MolGraph(i[1]))
    break

In [115]:
sys.argv = [""]
args = get_args()
args.device = "cuda:0"
gcn = GCN(args)
gcn.cuda()

In [122]:
X

In [111]:
gcn.forward(Y)[0]

RuntimeError: Expected object of device type cuda but got device type cpu for argument #2 'mat2' in call to _th_mm

In [21]:
opt = torch.nn.Linear(50, 50)

In [27]:
opt(gcn.forward(Y)[0])

tensor([[-0.0282,  0.0010,  0.1257,  ...,  0.0358,  0.0920,  0.0633],
        [-0.0633, -0.0360,  0.0435,  ...,  0.1123,  0.0942,  0.0485],
        [ 0.0159, -0.1049,  0.0004,  ..., -0.0007,  0.0266,  0.1323],
        ...,
        [-0.0834,  0.0271,  0.0630,  ...,  0.0840,  0.0499,  0.0218],
        [-0.1054, -0.0311,  0.0378,  ...,  0.0550,  0.0854,  0.0327],
        [-0.0654,  0.0254,  0.1890,  ...,  0.0316,  0.0437,  0.0630]],
       grad_fn=<AddmmBackward>)

In [123]:
from mol_opt import mol_opt
molopt = mol_opt.MolOpt(args)
molopt.cuda()

MolOpt(
  (GCN): GCN(
    (W_message_i): Linear(in_features=100, out_features=50, bias=False)
    (W_message_h): Linear(in_features=50, out_features=50, bias=False)
    (W_message_o): Linear(in_features=143, out_features=50, bias=True)
    (W_mol_h): Linear(in_features=50, out_features=100, bias=True)
    (W_mol_o): Linear(in_features=100, out_features=1, bias=True)
    (dropout_gcn): Dropout(p=0.0, inplace=False)
    (dropout_ffn): Dropout(p=0.0, inplace=False)
  )
  (opt): Linear(in_features=50, out_features=50, bias=True)
  (delta_loss): MSELoss()
)

In [124]:
molopt.forward_train(X, Y)

tensor(0.0283, device='cuda:0', grad_fn=<MeanBackward0>)

In [127]:
start = time.time()
for idx, i in enumerate(data_loader):
    X = (MolGraph(i[0]))
    Y = (MolGraph(i[1]))
    
    # create your optimizer
    optimizer = torch.optim.SGD(molopt.parameters(), lr=0.01)

    # in your training loop:
    optimizer.zero_grad()   # zero the gradient buffers
    loss = molopt.forward_train(X, Y)
    loss.backward()
    optimizer.step()    # Does the update

    print (idx, loss)
end = time.time()
print(end - start)

0 tensor(0.0185, device='cuda:0', grad_fn=<MeanBackward0>)
1 tensor(0.0178, device='cuda:0', grad_fn=<MeanBackward0>)
2 tensor(0.0169, device='cuda:0', grad_fn=<MeanBackward0>)
3 tensor(0.0167, device='cuda:0', grad_fn=<MeanBackward0>)
4 tensor(0.0177, device='cuda:0', grad_fn=<MeanBackward0>)
5 tensor(0.0181, device='cuda:0', grad_fn=<MeanBackward0>)
6 tensor(0.0178, device='cuda:0', grad_fn=<MeanBackward0>)
7 tensor(0.0176, device='cuda:0', grad_fn=<MeanBackward0>)
8 tensor(0.0174, device='cuda:0', grad_fn=<MeanBackward0>)
9 tensor(0.0179, device='cuda:0', grad_fn=<MeanBackward0>)
10 tensor(0.0181, device='cuda:0', grad_fn=<MeanBackward0>)
11 tensor(0.0179, device='cuda:0', grad_fn=<MeanBackward0>)
12 tensor(0.0172, device='cuda:0', grad_fn=<MeanBackward0>)
13 tensor(0.0179, device='cuda:0', grad_fn=<MeanBackward0>)
14 tensor(0.0177, device='cuda:0', grad_fn=<MeanBackward0>)
15 tensor(0.0173, device='cuda:0', grad_fn=<MeanBackward0>)
16 tensor(0.0164, device='cuda:0', grad_fn=<MeanBa

In [100]:
start = time.time()
for idx, i in enumerate(data_loader):
    X = (MolGraph(i[0]))
    Y = (MolGraph(i[1]))
    
    # create your optimizer
    optimizer = torch.optim.SGD(molopt.parameters(), lr=0.01)

    # in your training loop:
    optimizer.zero_grad()   # zero the gradient buffers
    loss = molopt.forward_train(X, Y)
    loss.backward()
    optimizer.step()    # Does the update

    print (idx, loss)
end = time.time()
print(end - start)

0 tensor(0.0183, grad_fn=<MeanBackward0>)
1 tensor(0.0179, grad_fn=<MeanBackward0>)
2 tensor(0.0165, grad_fn=<MeanBackward0>)
3 tensor(0.0164, grad_fn=<MeanBackward0>)
4 tensor(0.0176, grad_fn=<MeanBackward0>)
5 tensor(0.0180, grad_fn=<MeanBackward0>)
6 tensor(0.0176, grad_fn=<MeanBackward0>)
7 tensor(0.0175, grad_fn=<MeanBackward0>)
8 tensor(0.0173, grad_fn=<MeanBackward0>)
9 tensor(0.0177, grad_fn=<MeanBackward0>)
10 tensor(0.0180, grad_fn=<MeanBackward0>)
11 tensor(0.0174, grad_fn=<MeanBackward0>)
12 tensor(0.0171, grad_fn=<MeanBackward0>)
13 tensor(0.0179, grad_fn=<MeanBackward0>)
14 tensor(0.0175, grad_fn=<MeanBackward0>)
15 tensor(0.0173, grad_fn=<MeanBackward0>)
16 tensor(0.0163, grad_fn=<MeanBackward0>)
17 tensor(0.0170, grad_fn=<MeanBackward0>)
18 tensor(0.0167, grad_fn=<MeanBackward0>)
19 tensor(0.0166, grad_fn=<MeanBackward0>)
20 tensor(0.0173, grad_fn=<MeanBackward0>)
21 tensor(0.0168, grad_fn=<MeanBackward0>)
22 tensor(0.0175, grad_fn=<MeanBackward0>)
23 tensor(0.0162, gra

In [85]:
import time

In [99]:
start = time.time()
print("plm")
end = time.time()
print(end - start)

plm
0.0001800060272216797
